<a href="https://colab.research.google.com/github/asifahsaan/T2-Rossman-Sales-Prediction---Time-Series/blob/master/Rossmann_Store_Sales_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Asif Ahsan
# Thesis-1 (Base Paper)
# Rossmann's Store Sales

**Rossmann is Germany's second-largest drug store chain, with over 3,600 stores in Europe.**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Handle table-like data and matrices
import numpy as np
import pandas as pd
import math 
# Modelling Algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
from sklearn.metrics import accuracy_score

import keras
from keras import backend as K

# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# Configure visualisations
%matplotlib inline
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 8 , 6

In [ ]:
# Modelling Helpers
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Normalizer , scale
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
import pandas as pd
import io
df = pd.read_csv("/content/gdrive/MyDrive/Thesis Data/Dataset/train.csv/train.csv", low_memory=False,parse_dates = True)
store = pd.read_csv("/content/gdrive/MyDrive/Thesis Data/Dataset/store.csv", low_memory=False)
test = pd.read_csv("/content/gdrive/MyDrive/Thesis Data/Dataset/test.csv/test.csv", low_memory=False)

In [ ]:
new = df.copy()

In [ ]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [ ]:
# df.Date.unique()

**The dataset is from 1 January 2013 to 31 July 2015** , **i.e. approx. 2.5 yrs.**

In [ ]:
li = ["DayOfWeek" , "StateHoliday" , "SchoolHoliday"]

for i in li:
  print(i)
  print(df[i].unique())
  print("-----------------------")

DayOfWeek
[5 4 3 2 1 7 6]
-----------------------
StateHoliday
['0' 'a' 'b' 'c']
-----------------------
SchoolHoliday
[1 0]
-----------------------


** a = public holiday, b = Easter holiday, c = Christmas, 0 = None**

**Indicates if the (Store, Date) was affected by the closure of public schools**

In [ ]:
df.Store.nunique()

1115

In [ ]:
# df.info()

In [ ]:
# test.info()

In [ ]:
# store.info()

In [ ]:
# fig, (axis1,axis2) = plt.subplots(1,2,figsize=(20,4))
# sns.countplot(x='Open',hue='DayOfWeek', data=df, ax=axis1)
# sns.pointplot(x='Open',hue='DayOfWeek', data=df, ax=axis2)

**The store is mainly closed on day 7 which is Sunday**

In [ ]:
# df.describe()

In [ ]:
# Extract year, month and date
def extract1(x):
    return int(str(x)[:4])

def extract2(x):
    return int(str(x)[5:7])

def extract3(x):
    return (str(x)[:7])

In [ ]:
df['Date'] = df['Date'].apply(extract3)
test['Date'] = test['Date'].apply(extract3)
df['Year']  = df['Date'].apply(extract1)
df['Month'] = df['Date'].apply(extract2)
test['Year']  = test['Date'].apply(extract1)
test['Month'] = test['Date'].apply(extract2)
avgsales = df.groupby('Date')["Sales"].mean()
percentchngsales = df.groupby('Date')["Sales"].sum().pct_change()

**Separating (year + month) in the Date attribute**
AND
**Year and Month also in separate column**

**Extracting month and year feature from the date**

In [ ]:
# sns.factorplot(x="Date" ,y = "Sales" , data=df, kind="point", aspect=2,size=12)

In [ ]:
# sns.factorplot(x="Id" ,y = "Sales" , data=xgb_sub, kind="point", aspect=2,size=12)

**Heat-Map to show correlation b/w numerical attributes**

In [ ]:
# correlation_map = df[df.columns].corr()
# obj = np.array(correlation_map)
# obj[np.tril_indices_from(obj)] = False
# fig,ax= plt.subplots()
# fig.set_size_inches(9,9)
# sns.heatmap(correlation_map, mask=obj,vmax=.7, square=True,annot=True)

In [ ]:
# def plotter(StringA , StringB):
#   fig, axes = plt.subplots(2,1)
#   fig.set_size_inches(15, 10)
#   sns.barplot(x=StringA, y=StringB, data=df ,hue="DayOfWeek", ax = axes[0])
#   sns.boxplot(x=StringA, y=StringB, data=df ,hue="DayOfWeek", ax=axes[1])

In [ ]:
# plotter("Year" , "Sales")

In [ ]:
# plotter("Year" , "Customers")

**Encoding Stateholiday similarly**

In [ ]:
df["StateHoliday"] = df["StateHoliday"].map({0: 0, "0": 0, "a": 1, "b": 1, "c": 1})
test["StateHoliday"] = test["StateHoliday"].map({0: 0, "0": 0, "a": 1, "b": 1, "c": 1})

In [ ]:
df.StateHoliday.value_counts()

0    986159
1     31050
Name: StateHoliday, dtype: int64

In [ ]:
# labels = 'Not-Affected' , 'Affected'
# sizes = df.SchoolHoliday.value_counts()
# colors = ['gold', 'silver']
# explode = (0.1, 0.0)
# plt.pie(sizes, explode=explode, labels=labels, colors=colors,
#         autopct='%1.1f%%', shadow=True, startangle=180)
# plt.axis('equal')
# plt.title("Sales Affected by Schoolholiday or Not ?")
# plt.plot()
# fig=plt.gcf()
# fig.set_size_inches(6,6)
# plt.show()

In [ ]:
# df.SchoolHoliday.value_counts()

In [ ]:
# df["Sales"].plot(kind='hist',bins=100,xlim=(0,15000))

**0 is raised because most of the times store was closed**

# STORE DATASET

In [ ]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [ ]:
temp = []
for i in df.groupby('Store')[["Sales", "Customers"]].mean()["Sales"]:
  temp.append(i)
store["Sales"] = temp

In [ ]:
temp = []
for i in df.groupby('Store')[["Sales", "Customers"]].mean()["Customers"]:
  temp.append(i)
store["Customers"] = temp

In [ ]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Sales,Customers
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,3945.704883,467.646497
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",4122.991507,486.045648
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",5741.253715,620.286624
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN,8021.769639,1100.057325
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN,3867.110403,444.360934


In [ ]:
# labels = 'a' , 'b' , 'c' , 'd'
# sizes = store.StoreType.value_counts()
# colors = ['orange', 'green' , 'red' , 'pink']
# explode = (0.1, 0.0 , 0.15 , 0.0)
# plt.pie(sizes, explode=explode, labels=labels, colors=colors,
#         autopct='%1.1f%%', shadow=True, startangle=180)
# plt.axis('equal')
# plt.title("Distribution of different StoreTypes")
# plt.plot()
# fig=plt.gcf()
# fig.set_size_inches(6,6)
# plt.show()

In [ ]:
# def plotmaster(StringA , StringB):
#   fig, axes = plt.subplots(3,1)
#   fig.set_size_inches(12, 15)
#   sns.barplot(x=StringA, y=StringB, data=store ,hue="StoreType", ax = axes[0])
#   sns.boxplot(x=StringA, y=StringB, data=store ,hue="StoreType", ax=axes[1])
#   sns.violinplot(x=StringA, y=StringB, data=store, hue="StoreType" , ax=axes[2])

In [ ]:
# plotmaster("Assortment" , "Sales")

In [ ]:
# plotmaster("Promo2" , "Sales")

In [ ]:
# store.isnull().sum()

**stores which are opened on Sundays **

In [ ]:
df[(df.Open == 1) & (df.DayOfWeek == 7)]['Store'].unique()

array([  85,  122,  209,  259,  262,  274,  299,  310,  335,  353,  423,
        433,  453,  494,  512,  524,  530,  562,  578,  676,  682,  732,
        733,  769,  863,  867,  931,  948, 1045, 1081, 1097, 1099,  877])

In [ ]:
store = pd.read_csv("/content/gdrive/MyDrive/Thesis Data/Dataset/store.csv", low_memory=False,parse_dates = True)
# store = pd.read_csv(io.StringIO(uploaded['store.csv'].decode('utf-8')))

In [ ]:
test = pd.read_csv("/content/gdrive/MyDrive/Thesis Data/Dataset/test.csv/test.csv", low_memory=False,parse_dates = True)
# test = pd.read_csv(io.StringIO(uploaded['test.csv'].decode('utf-8')))

In [ ]:
df = new.copy()

In [ ]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,log_sales,...,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,year_2013,year_2014,year_2015
0,1,5,2015-07-31,5263,555,1,1,0.0,1,8.568456,...,0,0,1,0,1,0,0,0,0,1
1,1,4,2015-07-30,5020,546,1,1,0.0,1,8.521185,...,0,0,1,0,1,0,0,0,0,1
2,1,3,2015-07-29,4782,523,1,1,0.0,1,8.472614,...,0,0,1,0,1,0,0,0,0,1
3,1,2,2015-07-28,5011,560,1,1,0.0,1,8.519391,...,0,0,1,0,1,0,0,0,0,1
4,1,1,2015-07-27,6102,612,1,1,0.0,1,8.716372,...,0,0,1,0,1,0,0,0,0,1


In [ ]:
test.fillna(1, inplace=True)
#  keep only open stores and sales > 1
df = df[df["Open"] != 0]
df = df[df["Sales"] > 0]
df['log_sales'] = np.log(df['Sales'])
df = pd.merge(df, store, on='Store')
test = pd.merge(test, store, on='Store')
df.fillna(0,inplace=True)
test.fillna(0,inplace=True)

In [ ]:
df["StateHoliday"] = df["StateHoliday"].map({0: 0, "0": 0, "a": 1, "b": 1, "c": 1})
test["StateHoliday"] = test["StateHoliday"].map({0: 0, "0": 0, "a": 1, "b": 1, "c": 1})
df['StateHoliday'] = df['StateHoliday'].astype(float)
test['StateHoliday'] = test['StateHoliday'].astype(float)

In [ ]:
new.head().Date

0    2015-07-31
1    2015-07-31
2    2015-07-31
3    2015-07-31
4    2015-07-31
Name: Date, dtype: object

In [ ]:
new.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


**Again extracting day , month and year**

In [ ]:
df["year"]=df.Date.apply(extract1)
df["month"]=df.Date.apply(extract2)

In [ ]:
df["Day"]=df.Date.apply(lambda x: int(str(x)[8:10]))

In [ ]:
test["year"]=test.Date.apply(extract1)
test["month"]=test.Date.apply(extract2)
test["Day"]=test.Date.apply(lambda x: int(str(x)[8:10]))

**Now getting dummies**

In [ ]:
df = pd.get_dummies(df,columns=['StoreType','Assortment','year'])
test = pd.get_dummies(test,columns=['StoreType','Assortment','year'])
test['year_2013']=0
test['year_2014']=0

In [ ]:
# df = new.copy()

In [ ]:
X = df.drop(['Sales','log_sales','Store','Date','Customers','CompetitionOpenSinceYear','Promo2SinceYear','PromoInterval'] , axis = 1)
y = df['log_sales']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=7)
X_test = test.drop(['Id','Store','Date' ,'CompetitionOpenSinceYear','Promo2SinceYear','PromoInterval'] , axis = 1)

In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(X_train , y_train)
pred = reg.predict(X_val)

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_val,pred))
print(rmse)

0.3731435568682413


## Note : 
## All the grid searchs are performed piecewise and looked for locally best parameters and thereafter shifted to the new parameters accordingly to make it computationally less expensive .

In [ ]:
from sklearn.model_selection import GridSearchCV
model = linear_model.Ridge(random_state = 1)

param_grid = {
"alpha" : [0.01 , 0.2 , 0.250 , 0.3] 
}
grid = GridSearchCV( model , param_grid , cv = 7 , scoring = "neg_mean_squared_error")
grid.fit(X,y)

GridSearchCV(cv=7, estimator=Ridge(random_state=1),
             param_grid={'alpha': [0.01, 0.2, 0.25, 0.3]},
             scoring='neg_mean_squared_error')

In [ ]:
grid.best_score_

-0.14055926917096614

In [ ]:
rmse = np.sqrt(-grid.best_score_)
print(rmse)

0.37491234865094286


**Even after repeated PIECE WISE gridsearchCV the min RMSE is 0.3749 which is still greater than linear regression using Ordinary Least Squares**

In [ ]:
grid.best_params_

{'alpha': 0.3}

In [ ]:
model = linear_model.Lasso(random_state = 1)

param_grid = {
"alpha" : [0.001 , 0.0001] ,
"max_iter" : [600 , 700]
}
grid = GridSearchCV( model , param_grid , cv = 7 , scoring = "neg_mean_squared_error")
grid.fit(X,y)

KeyboardInterrupt: ignored

**Though alpha is checked for 10^-4 learning rate which is very low and very slow to train but the initial piece-wise checkings with LASSO favoured low learning rates so we have to check them .**

In [ ]:
grid.best_score_

In [ ]:
rmse = np.sqrt(-grid.best_score_)
print(rmse)

In [ ]:
grid.best_params_

**The result is slightly impressive than Ridge but still slightly worse than ordinary sqaures one for the best_params_ as evaluated**



## Ensemble Techniques

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100,max_depth=15)

rf.fit(X_train,y_train)

rf_pred = rf.predict(X_val)

In [ ]:
rmse_rf = np.sqrt(mean_squared_error(y_val,rf_pred))
rmse_rf

## Decision Tree

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeRegressor()
clf.fit(X_train , y_train)
pred = clf.predict(X_val)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_val,pred))
print(rmse)

**Without parameter tuning the decision tree regressor is giving 0.1816 as RMSE which is quite good in comparison with Lasso , Ridge , etc.  But above training was not cross-validated as Lasso and Ridge were. **

## The parameters in the XGBRegressor have been tuned and only showed one of the best here.

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(max_depth=15,n_jobs=4,n_estimators=120,subsample=0.7)
xgb.fit(X_train,y_train)
xg_pred = xgb.predict(X_val)
rmse_xgb = np.sqrt(mean_squared_error(y_val,xg_pred))
rmse_xgb

## XGBoost Regressor achieved a considerably low RMSE of 0.10874 and did really well in this case.

In [ ]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
xgb = XGBRegressor(max_depth=15,n_jobs=4,n_estimators=120,subsample=0.7)
scores = cross_val_score(xgb, X , y , cv=5)
scores

## Cross validations upto 5 also yielded a minimum of 0.1205  which is less than any other model's RMSE. 

**Now training on full data and predicting using new trained model**

In [ ]:
from xgboost import XGBRegressor
xgb_final = XGBRegressor(max_depth=15,n_jobs=4,n_estimators=120,subsample=0.7)
xgb_final.fit(X,y)

In [ ]:
ranks = np.argsort(-xgb_final.feature_importances_)
f, ax = plt.subplots(figsize=(15, 10))

sns.barplot(x=xgb_final.feature_importances_[ranks], y=X_train.columns.values[ranks], orient='h')
ax.set_xlabel("Importance Of Features")
plt.tight_layout()
plt.show()

In [ ]:
X_Test = X_test.copy()

In [ ]:
a = X_test["year_2015"]

In [ ]:
X_test.drop("year_2015" , axis = 1 , inplace = True)

In [ ]:
X_test["year_2015"] = a

In [ ]:
X_test.columns

In [ ]:
final_predictions = xgb_final.predict(X_test)

In [ ]:
np.exp(final_predictions)

In [ ]:
dic = {
    'Id':test['Id'].astype(int),
    'DayOfWeek': test['DayOfWeek'],
    'Day': test['Day'],
    'month': test['month'],
    'Sales': np.exp(final_predictions)
}

xgb_sub = pd.DataFrame(dic ,columns=['Id','DayOfWeek','Day','month','Sales'])

In [ ]:
xgb_sub.head()

In [ ]:
preddf = xgb_sub

In [ ]:
preddf.month.unique()

In [ ]:
preddf.DayOfWeek.unique()

In [ ]:
preddfg = preddf.groupby(['DayOfWeek'])['Sales'].median()
# selected_sales = subs.groupby(['Year', 'Month'])['Sales'].median()

In [ ]:
preddfg.head()

In [ ]:
x = preddf['DayOfWeek']
y = preddf['Sales']
plt.bar(x,y)
plt.show()

In [ ]:
# ssum = preddf.groupby(['DayOfWeek'])['Sales'].median()
preddf['SalesPerDay'] = preddf.groupby('Day')['Sales'].sum()
preddf.head()

In [ ]:
preddfgs = preddf.groupby('DayOfWeek')
preddfgs.first()

In [ ]:
x = preddf['DayOfWeek']
y = preddf['Sales']

print(x)

In [ ]:
ids = xgb_sub['Id']
sales = xgb_sub['Sales']

In [ ]:
plt.figure(figsize=(10,5))
plt.hist(x=xgb_sub.Sales, bins=30,color = "green")
plt.ylabel('number of observations')
plt.xlabel('daily sales in $')
plt.title('Sales Distribution')

In [ ]:
# # Plot the ID and Sales columns using Matplotlib
# plt.plot(ids, sales)

# # Add a title and axis labels
# plt.title('ID vs Sales')
# plt.xlabel('ID')
# plt.ylabel('Sales')

# # Display the plot
# plt.show()

In [ ]:
# # Bar chart
# plt.bar(ids, sales)

In [ ]:
# # Scatter plot
# plt.scatter(ids, sales)

In [ ]:
xgb_sub.head()

In [ ]:
result = xgb_sub.sort_values(by='Id',ascending=True)
result.to_csv('result.csv',index=False)

In [ ]:
result.head()

In [ ]:
from google.colab import files
files.download('result.csv')